In [ ]:
# !pip install datasets opencv-contrib-python opencv-python ipywidgets scipy

## Implementing Bag of Visual Words

### Training Dataset Load and Cleaning

First, we want to import a dataset of images to train the model.

Feel free to use any images you like, but, if you’d like to follow along with the same images, you can download them using HuggingFace Datasets.

In [ ]:
from datasets import load_dataset

# download the dataset
data = load_dataset(
    'frgfm/imagenette',
    '160px',
    'verification_mode=no_checks', # set to 'verification_mode=all_checks' if seeing splits Error
    split='train'
)
data

The chosen dataset is composed of $9469$ images, representing dogs, radios, fishes, cities, and so on. We can visualise some of them below.

In [ ]:
# important to use imagenet[0]['image'] rather than imagenet['image'][0]
# as the latter loads the entire image ctrtvgvrtttttolumn then extracts index 0 (it's slow)
data[3900]['image']

In [ ]:
data[5020]['image']

We can now transform these images from PIL objects to numpy arrays.

In [ ]:
import numpy as np
from tqdm.notebook import tqdm

# generate an array from the original dataset
images_training = []

for n in tqdm(range(0,len(data))):
    images_training.append(np.array(data[n]['image']))

To simplify the model and speed-up the process, we can transform the images from *RGB* to grayscale. This will reduce the image dimension from $3$ to $2$.

To do that, we can use ``cv2``. Note that the dataset might include a mix of RGB and grayscale images. In that case, we would need to trasform only those in RGB, while keeping the grayscale ones as they are. The below code addresses this by looking at the shape's length: if the shape equals $2$, then the image is in grayscale, if the shape is $3$, then the image is RGB.

In [ ]:
import cv2

# convert images to grayscale
bw_images = []
for img in tqdm(images_training):
    # if RGB, transform into grayscale
    if len(img.shape) == 3:
        ### START YOUR CODE HERE
        bgr2bw = None
        bw_images.append(bgr2bw)
        ### END YOUR CODE HERE
    else:
        # if grayscale, do not transform
        bw_images.append(img)

Let's take a look at a transformed image.

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(bw_images[1], cmap='gray')
plt.show()

We have now a dataset of images that we can use to train our model. Let's see how to build it.

### Visual features Extraction

The first step is to extract the image visual features (keypoints and descriptors). As previously mentioned, we will use SIFT, a feature detector.

In [ ]:
### START YOUR CODE HERE
# defining feature extractor that we want to use (SIFT)
extractor = None

# initialize lists where we will store *all* keypoints and descriptors
keypoints = []
descriptors = []

for img in tqdm(bw_images):
    # extract keypoints and descriptors for each image
    img_keypoints, img_descriptors = None
    keypoints.append(img_keypoints)
    descriptors.append(img_descriptors)
### END YOUR CODE HERE

Some images might return `None` when attempting to extract keypoints using SIFT. This is (usually) because the SIFT algorithm cannot detect any keypoints (common when an image is very flat without obvious edges). In that case, we remove those images from our data.

In [ ]:
print(f"len before: {len(descriptors)}")
# initialize list to store idx values of records to drop
to_drop = []
for i, img_descriptors in enumerate(descriptors):
    # if there are no descriptors, add record idx to drop list
    if img_descriptors is None:
        to_drop.append(i)

print(f"indexes: {to_drop}")
# delete from list in reverse order
for i in sorted(to_drop, reverse=True):
    del descriptors[i], keypoints[i]

print(f"len after: {len(descriptors)}")

For this sample, we fortunately don't have this issue - as we can see the before/after length has not changed.

Now that we have extracted the features, we can visualize them on some images.

In [ ]:
output_image = []
for x in range(3):
    output_image.append(cv2.drawKeypoints(bw_images[x], keypoints[x], 0, (255, 0, 0),
                                 flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS))
    plt.imshow(output_image[x], cmap='gray')
    plt.show()

The centre of each circle is the keypoint location, the line from the centre to the circle is the orientation of the keypoint, and the size of the circle is the scale at which this feature was detected.

We are now ready to build the codebook!

### Building the Codebook

To build the codebook, we will randomly sample $1000$ of the image descriptors. To create the sample we use Numpy to generate $1000$ random integer numbers from $0$ to $9469$ (the length of the dataset). We set `np.random.seed(0)` to generate the same sample for reproducability.

In [ ]:
import numpy as np

# select the same numbers in each run
np.random.seed(0)
# select 1000 random image index values
sample_idx = np.random.randint(0, len(data)+1, 1000).tolist()
len(sample_idx)

First, we want to stack the generated descriptors into a numpy array. This will be a single array of *all* the chosen descriptors across *all* the corresponding images. To be more precise, we are going to extract the corresponding keypoints, even though we are not going to use them going ahead.

In [ ]:
# extract the sample from descriptors
# (we don't need keypoints)
descriptors_sample = []

for n in tqdm(sample_idx):
    descriptors_sample.append(np.array(descriptors[n]))

In [ ]:
all_descriptors = []
# extract image descriptor lists
for img_descriptors in tqdm(descriptors_sample):
    # extract specific descriptors within the image
    for descriptor in img_descriptors:
        all_descriptors.append(descriptor)

### START YOUR CODE HERE        
# convert to single numpy array
all_descriptors = None
### END YOUR CODE HERE

In [ ]:
# check the shape
all_descriptors.shape

We now have *all* descriptors (each descriptor being a 128-dimensional vector) across *all* sampled images. There are around *1,130,000* of these in total.

In [ ]:
# we can count the number of descriptors contained in descriptors to confirm
count = []
for img_descriptors in descriptors_sample:
    count.append(len(img_descriptors))
# here we can see the number of descriptors for the first five images
print(f"first five: {count[:5]}")
# and if we sum them all, we should see the 39893 from before
print(f"count all: {sum(count)}")

We now want to group similar visual features (descriptors) using *k-means*. After a few tests, we chose $k=200$ for our model.

After k-means, all images will be reduced to *visual words* and we will use that to build our codebook.

*It can be difficult to find the optimal size of our codebook - if too small, visual words could be not representative of all image regions, if too large, there could be too many visual words with little-no of them being shared between images (making comparisons very hard or impossible).*

Once built, the codebook won't change anymore, i.e., will stay fixed!

In [ ]:
# perform k-means clustering to build the codebook
from scipy.cluster.vq import kmeans

### START YOUR CODE HERE
k = 200
iters = 1
codebook, variance = None
### END YOUR CODE HERE

### Saving Codebook

We save the codebook and key parameters like `k` into a file so that in the future we just load these when processing new images.

In [ ]:
import joblib

# save number of clusters and codebook
# Joblib dumps Python object into one file
joblib.dump((k, codebook), "bovw-codebook.pkl", compress=3)

To *emulate* this approach, we can go ahead and load `k` and our `codebook` from file to use for the rest of the notebook - this is optional and not necessary, feel free to not run the next code cell.

In [ ]:
# load the visual features, number of clusters, and codebook
k, codebook = joblib.load("bovw-codebook.pkl")

### Building Sparse Vectors

After building our `codebook` we can begin building the sparse vector representations of our images, this requires three steps, (1) vector quantization, (2) frequency count, and (3) tf-idf.

#### 1. Vector quantization:

Transforming the visual *feature* representations of our images into *visual words*.

In [ ]:
# vector quantization
from scipy.cluster.vq import vq

visual_words = []
for img_descriptors in tqdm(descriptors):
    ### START YOUR CODE HERE
    # for each image, map each descriptor to the nearest codebook entry
    img_visual_words, distance = None
    visual_words.append(img_visual_words)
    ### END YOUR CODE HERE

In [ ]:
# let's see what the visual words look like for image 0
visual_words[0][:5], len(visual_words[0])

*each of these values, `84`, `22`, etc., represent a centroid (visual word) from the codebook:*

In [ ]:
# the centroid that represents visual word 84 is of dimensionality...
codebook[84].shape  # (all have the same dimensionality)

#### 2. Frequency count

For each visual word found in the codebook, how often does it appear in an image? We format this as a sparse vector with frequency counts. We select the position to place a count for a visual word using the visual word identifier, e.g., position `84` for the visual word we showed above.

In [ ]:
frequency_vectors = []
for img_visual_words in tqdm(visual_words):
    ### START YOUR CODE HERE
    # create a frequency vector for each image
    img_frequency_vector = None
    for word in img_visual_words:
        img_frequency_vector[word] = None   # add 1 more
    frequency_vectors.append(img_frequency_vector)
    ### END YOUR CODE HERE
# stack together in numpy array
frequency_vectors = np.stack(frequency_vectors)

In [ ]:
frequency_vectors.shape

*we know from above that positions `[84,  22,  45, 172]` should all count at least `1` (and at least `2` for position `172`) in the frequency vector for image 0:*

In [ ]:
# we know from above that ids 84, 22, 45, and 172 appear in image 0
for i in [84,  22,  45, 172]:
    print(f"{i}: {frequency_vectors[0][i]}")

*let's see some more of image 0's frequency vector...*

In [ ]:
frequency_vectors[0][:20]

*we can also visualize the full image in a histogram like so:*

In [ ]:
plt.bar(list(range(k)), frequency_vectors[0])
plt.show()

#### 3. Tf-idf

The above histogram does not consider the relevance of the visual words. That is why, we want to re-weight them using tf-idf.

For reference, here is the formula. Try and refer to this if any of the following code is hard to understand.

$$tf\textrm{--}idf_{t,d} = tf_{t,d} * idf_t = tf_{t,d} * log\frac{N}{df_t}$$

where:

* $tf_{t,d}$ is the term frequency of the visual word $t$ in the image $d$ (the number of times $t$ occurs in $d$).
* $N$ is the total number of images.
* $df_t$ number of images containing visual word $t$.
* $log\frac{N}{df_t}$ measures the how common visual word $t$ is across all images in the database. This is low if the visual word $d$ occurs many times in the image, high otherwise.

Let's first calculate "*df*", given $N = 9469$, i.e., the length of our dataset.

In [ ]:
# N is the number of images, i.e. the size of the dataset
N = 9469

### START YOUR CODE HERE
# df is the number of images that a visual word appears in
# we calculate it by counting non-zero values as 1 and summing
df = np.sum(frequency_vectors > 0, axis=0)
### END YOUR CODE HERE

In [ ]:
df.shape, df[:5]

*from this we know that visual word `0` appears in `7935` images, visual word `1` in `7373` images, and so on.*

Using `N` and `df` we can calculate *"idf"* (which is the same for every image).

In [ ]:
### START YOUR CODE HERE
idf = None
idf.shape, idf[:5]
### END YOUR CODE HERE

Now we move on to the full tf-idf calculation, we just multiply the *"tf"* for each image by this `idf` vector. We have already calculated the *"tf"* values, they are the frequency vectors stored in `frequency_vectors`.

In [ ]:
### START YOUR CODE HERE
tfidf = None
tfidf.shape, tfidf[0][:5]
### END YOUR CODE HERE

Visualizing the tf-idf vector for image `0` gives use:

In [ ]:
plt.bar(list(range(k)), tfidf[0])
plt.show()

### Finalizing Results

We have now generated tf-idf vectors based on the visual words of our images. To compare these images we can use a metric like *cosine similarity*.

We will use image `6` as our search term:

In [ ]:
search_i = 0

plt.imshow(bw_images[search_i], cmap='gray')
plt.show()

Again, for reference, our cosine similarity range is $[0,1]$, and it is calculated as follows:

$$cossim(A,B)= cos(\theta)=\frac{A \cdot B}{||A|| \space ||B||}$$

In [ ]:
# cosine similarity
from numpy.linalg import norm

### START YOUR CODE HERE
a = tfidf[search_i]
b = tfidf  # set search space to the full sample

cosine_similarity = None
print("Min cosine similarity:", round(np.min(cosine_similarity),1))
print("Max cosine similarity:", np.max(cosine_similarity))
### END YOUR CODE HERE

In our TF-IDF vectors, two highly similar images should be separated by a very small angular distance. Using cosine similarity, small angles output a high score (i.e., towards $1$).

Let's pick a random image from our dataset and retrieve the top 5 most similar images.

In [ ]:
cosine_similarity.shape

In [ ]:
cosine_similarity

To find the `top_k` most similar items, we use Numpy's `argpartition`.

In [ ]:
### START YOUR CODE HERE
top_k = 5
idx = None
idx
### END YOUR CODE HERE

The first image in the array should correspond to the image itself. Therefore, the cosine similarity must be $≅1$.

In [ ]:
cosine_similarity[idx[0]]

In [ ]:
for i in idx:
    print(f"{i}: {round(cosine_similarity[i], 4)}")
    plt.imshow(bw_images[i], cmap='gray')
    plt.show()

This looks promising, we can test with a few more images.

In [ ]:
def search(i: int, top_k: int = 5):
    print("Search image:")
    # show the search image
    plt.imshow(bw_images[i], cmap='gray')
    plt.show()
    print("-----------------------------------------------------")
    # get search image vector
    a = tfidf[i]

    ### START YOUR CODE HERE
    # get the cosine distance for the search image `a`
    cosine_similarity = None
    # get the top k indices for most similar vecs
    idx = None
    ### END YOUR CODE HERE

    # display the results
    for i in idx:
        print(f"{i}: {round(cosine_similarity[i], 4)}")
        plt.imshow(bw_images[i], cmap='gray')
        plt.show()

In [ ]:
search(1200)

In [ ]:
search(2)

In [ ]:
search(8450)

The results are not always perfect, but for the most part it seems to manage in returning similar images.

---